In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date

In [4]:
sla_df = pd.read_excel('./SLA_matrix.xlsx')
sla_df

,1st Attempt SLA\n(Working Days),Unnamed: 1,Destination (Buyer),Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,Metro Manila,Luzon,Visayas,Mindanao
1,Origin\n(Seller),Metro Manila,3 working days,5 working days,7 working days,7 working days
2,NaN,Luzon,5 working days,5 working days,7 working days,7 working days
3,NaN,Visayas,7 working days,7 working days,7 working days,7 working days
4,NaN,Mindanao,7 working days,7 working days,7 working days,7 working days
5,NaN,NaN,NaN,NaN,NaN,NaN
6,"Working Days are defined as Mon - Sat, Excludi...",NaN,NaN,NaN,NaN,NaN
7,SLA calculation begins from the next day after...,NaN,NaN,NaN,NaN,NaN
8,2nd Attempt must be no later than 3 working da...,NaN,NaN,NaN,NaN,NaN


In [5]:
orders_df = pd.read_csv('./delivery_orders_march.csv')
orders_df

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...
...,...,...,...,...,...,...
3176308,31504087640510,1585821728,1.585978e+09,NaN,"mayapis. edeliza JC274 5542 brgy.Dita, Metro M...",Intimate St) compound. Sekiat 2B8 98-B Mel far...
3176309,31504147352227,1585895011,1.586179e+09,1.586232e+09,"Superdome PORKY K307 Libertad, PCGCC sub.bagui...","13th 3616 Pilipinas, :1v26 1f-5 (Village) bl10..."
3176310,31504462290482,1585826951,1.585965e+09,NaN,Galay's 1republic 333-18 Diliman 74d Malbay fe...,"Lane, 379B 1739A Ivc AFPFC Cainta, Waterpark, ..."
3176311,31504851495943,1585891194,1.586230e+09,NaN,"#4560, 80south biloca vi!!age KM21, MANRESA, ,...",lrt-2 Consolacion 224-A Paradise 6D 36st champ...


# Convert unix time to Phillipines local date and time

In [7]:
def convert_to_local_datetime(dt_series):
    return dt_series.dt.tz_localize("GMT").dt.tz_convert('Asia/Manila').dt.tz_localize(None)

In [9]:
orders_df['pick'] = convert_to_local_datetime(orders_df['pick'])
orders_df['1st_deliver_attempt'] = convert_to_local_datetime(orders_df['1st_deliver_attempt'])
orders_df['2nd_deliver_attempt'] = convert_to_local_datetime(orders_df['2nd_deliver_attempt'])
orders_df

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,2020-03-02 16:39:57,2020-03-05 13:09:18,NaT,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,2020-03-04 16:19:28,2020-03-06 10:53:56,2020-03-10 08:02:56,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,2020-03-04 15:20:34,2020-03-06 09:56:19,NaT,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,2020-03-05 22:36:56,2020-03-07 12:45:41,NaT,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,2020-03-04 18:38:25,2020-03-06 15:41:40,NaT,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...
...,...,...,...,...,...,...
3176308,31504087640510,2020-04-02 18:02:08,2020-04-04 13:22:22,NaT,"mayapis. edeliza JC274 5542 brgy.Dita, Metro M...",Intimate St) compound. Sekiat 2B8 98-B Mel far...
3176309,31504147352227,2020-04-03 14:23:31,2020-04-06 21:17:27,2020-04-07 11:52:11,"Superdome PORKY K307 Libertad, PCGCC sub.bagui...","13th 3616 Pilipinas, :1v26 1f-5 (Village) bl10..."
3176310,31504462290482,2020-04-02 19:29:11,2020-04-04 09:55:40,NaT,Galay's 1republic 333-18 Diliman 74d Malbay fe...,"Lane, 379B 1739A Ivc AFPFC Cainta, Waterpark, ..."
3176311,31504851495943,2020-04-03 13:19:54,2020-04-07 11:19:12,NaT,"#4560, 80south biloca vi!!age KM21, MANRESA, ,...",lrt-2 Consolacion 224-A Paradise 6D 36st champ...


# Define a function to query for SLA from city pair

In [18]:
sld = {('manila', 'manila'): 3, ('luzon', 'manila'): 5, ('manila', 'luzon'): 5, ('luzon', 'luzon'): 5}
def get_sla(city_pair):
    return sld.get(city_pair, 7)

# Define a function that calculates the number of days to be counted (excluding Sundays and Public Holidays)

In [14]:
# ph = [date(2020, 3, 8), date(2020, 3, 25), date(2020, 3, 30), date(2020, 3, 31)]
ph = [date(2020, 3, 25), date(2020, 3, 30), date(2020, 3, 31)]
ph

[datetime.date(2020, 3, 25),
 datetime.date(2020, 3, 30),
 datetime.date(2020, 3, 31)]

In [15]:
def get_days(pick, deli):
    days = deli - pick
    count = 0
    if not pick > ph[-1] and not deli < ph[0]:
        for date in ph:
            if date > pick and date < deli:
                count +=1
    
    days_from_sun = 6 - pick.weekday()
    next_sunday = pick + timedelta(days=days_from_sun)
    
    while next_sunday < deli:
        count += 1
        next_sunday = next_sunday + timedelta(days=7)
    
    return days.days - count

# Extract out city names from addresses, then query for SLA

In [16]:
orders_df['buyeraddress'] = orders_df['buyeraddress'].str.lower()
orders_df['selleraddress'] = orders_df['selleraddress'].str.lower()
orders_df['origin'] = orders_df['buyeraddress'].str.split().str[-1]
orders_df['dest'] = orders_df['selleraddress'].str.split().str[-1]
orders_df

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,origin,dest
0,2215676524,2020-03-02 16:39:57,2020-03-05 13:09:18,NaT,"baging ldl buenavista,patag.cagayan buagsong,c...",pantranco vill. 417 warehouse# katipunan 532 (...,manila,manila
1,2219624609,2020-03-04 16:19:28,2020-03-06 10:53:56,2020-03-10 08:02:56,coloma's quzom casanas site1 masiyan 533a stol...,"bldg 210a moras c42b 2b16,168 church) complex ...",manila,manila
2,2220979489,2020-03-04 15:20:34,2020-03-06 09:56:19,NaT,"21-o lumangdaan,capitangan,abucay,bataan .bign...","#66 150-c, drive, milagros joe socorro metro m...",manila,manila
3,2221066352,2020-03-05 22:36:56,2020-03-07 12:45:41,NaT,"616espiritu martinville,manuyo #5paraiso kengi...","999maii 201,26 villaruel barretto gen.t number...",manila,manila
4,2222478803,2020-03-04 18:38:25,2020-03-06 15:41:40,NaT,l042 summerbreezee1 l2(balanay analyn lot760 c...,g66manila hiyas fitness maysilo magdiwang lt.4...,luzon,manila
...,...,...,...,...,...,...,...,...
3176308,31504087640510,2020-04-02 18:02:08,2020-04-04 13:22:22,NaT,"mayapis. edeliza jc274 5542 brgy.dita, metro m...",intimate st) compound. sekiat 2b8 98-b mel far...,manila,manila
3176309,31504147352227,2020-04-03 14:23:31,2020-04-06 21:17:27,2020-04-07 11:52:11,"superdome porky k307 libertad, pcgcc sub.bagui...","13th 3616 pilipinas, :1v26 1f-5 (village) bl10...",manila,manila
3176310,31504462290482,2020-04-02 19:29:11,2020-04-04 09:55:40,NaT,galay's 1republic 333-18 diliman 74d malbay fe...,"lane, 379b 1739a ivc afpfc cainta, waterpark, ...",manila,manila
3176311,31504851495943,2020-04-03 13:19:54,2020-04-07 11:19:12,NaT,"#4560, 80south biloca vi!!age km21, manresa, ,...",lrt-2 consolacion 224-a paradise 6d 36st champ...,luzon,manila


In [19]:
orders_df['sla'] = orders_df.apply(lambda row: get_sla((row.origin, row.dest)), axis=1)
orders_df

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,origin,dest,sla
0,2215676524,2020-03-02 16:39:57,2020-03-05 13:09:18,NaT,"baging ldl buenavista,patag.cagayan buagsong,c...",pantranco vill. 417 warehouse# katipunan 532 (...,manila,manila,3
1,2219624609,2020-03-04 16:19:28,2020-03-06 10:53:56,2020-03-10 08:02:56,coloma's quzom casanas site1 masiyan 533a stol...,"bldg 210a moras c42b 2b16,168 church) complex ...",manila,manila,3
2,2220979489,2020-03-04 15:20:34,2020-03-06 09:56:19,NaT,"21-o lumangdaan,capitangan,abucay,bataan .bign...","#66 150-c, drive, milagros joe socorro metro m...",manila,manila,3
3,2221066352,2020-03-05 22:36:56,2020-03-07 12:45:41,NaT,"616espiritu martinville,manuyo #5paraiso kengi...","999maii 201,26 villaruel barretto gen.t number...",manila,manila,3
4,2222478803,2020-03-04 18:38:25,2020-03-06 15:41:40,NaT,l042 summerbreezee1 l2(balanay analyn lot760 c...,g66manila hiyas fitness maysilo magdiwang lt.4...,luzon,manila,5
...,...,...,...,...,...,...,...,...,...
3176308,31504087640510,2020-04-02 18:02:08,2020-04-04 13:22:22,NaT,"mayapis. edeliza jc274 5542 brgy.dita, metro m...",intimate st) compound. sekiat 2b8 98-b mel far...,manila,manila,3
3176309,31504147352227,2020-04-03 14:23:31,2020-04-06 21:17:27,2020-04-07 11:52:11,"superdome porky k307 libertad, pcgcc sub.bagui...","13th 3616 pilipinas, :1v26 1f-5 (village) bl10...",manila,manila,3
3176310,31504462290482,2020-04-02 19:29:11,2020-04-04 09:55:40,NaT,galay's 1republic 333-18 diliman 74d malbay fe...,"lane, 379b 1739a ivc afpfc cainta, waterpark, ...",manila,manila,3
3176311,31504851495943,2020-04-03 13:19:54,2020-04-07 11:19:12,NaT,"#4560, 80south biloca vi!!age km21, manresa, ,...",lrt-2 consolacion 224-a paradise 6d 36st champ...,luzon,manila,5


# Decide whether an order is late on first delivery attempt

In [20]:
orders_df['days_first'] = orders_df.apply(lambda row: get_days(row['pick'], row['1st_deliver_attempt']), axis=1)
orders_df['ontime_first'] = orders_df['days_first'] <= orders_df['sla']
orders_df

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,origin,dest,sla,days_first,ontime_first
0,2215676524,2020-03-02 16:39:57,2020-03-05 13:09:18,NaT,"baging ldl buenavista,patag.cagayan buagsong,c...",pantranco vill. 417 warehouse# katipunan 532 (...,manila,manila,3,2,True
1,2219624609,2020-03-04 16:19:28,2020-03-06 10:53:56,2020-03-10 08:02:56,coloma's quzom casanas site1 masiyan 533a stol...,"bldg 210a moras c42b 2b16,168 church) complex ...",manila,manila,3,1,True
2,2220979489,2020-03-04 15:20:34,2020-03-06 09:56:19,NaT,"21-o lumangdaan,capitangan,abucay,bataan .bign...","#66 150-c, drive, milagros joe socorro metro m...",manila,manila,3,1,True
3,2221066352,2020-03-05 22:36:56,2020-03-07 12:45:41,NaT,"616espiritu martinville,manuyo #5paraiso kengi...","999maii 201,26 villaruel barretto gen.t number...",manila,manila,3,1,True
4,2222478803,2020-03-04 18:38:25,2020-03-06 15:41:40,NaT,l042 summerbreezee1 l2(balanay analyn lot760 c...,g66manila hiyas fitness maysilo magdiwang lt.4...,luzon,manila,5,1,True
...,...,...,...,...,...,...,...,...,...,...,...
3176308,31504087640510,2020-04-02 18:02:08,2020-04-04 13:22:22,NaT,"mayapis. edeliza jc274 5542 brgy.dita, metro m...",intimate st) compound. sekiat 2b8 98-b mel far...,manila,manila,3,1,True
3176309,31504147352227,2020-04-03 14:23:31,2020-04-06 21:17:27,2020-04-07 11:52:11,"superdome porky k307 libertad, pcgcc sub.bagui...","13th 3616 pilipinas, :1v26 1f-5 (village) bl10...",manila,manila,3,2,True
3176310,31504462290482,2020-04-02 19:29:11,2020-04-04 09:55:40,NaT,galay's 1republic 333-18 diliman 74d malbay fe...,"lane, 379b 1739a ivc afpfc cainta, waterpark, ...",manila,manila,3,1,True
3176311,31504851495943,2020-04-03 13:19:54,2020-04-07 11:19:12,NaT,"#4560, 80south biloca vi!!age km21, manresa, ,...",lrt-2 consolacion 224-a paradise 6d 36st champ...,luzon,manila,5,2,True


# Extract out orders that are late on first delivery attempt. Label them as late.

In [21]:
orders_df_late = orders_df[~orders_df['ontime_first']]
result_df = pd.DataFrame({'orderid': orders_df_late['orderid'], 'is_late': 1})
result_df

,orderid,is_late
7,2224695304,1
21,2227816770,1
24,2227831996,1
29,2227849732,1
38,2227878093,1
...,...,...
3175389,31104513808320,1
3175410,31111243311095,1
3175414,31111809571977,1
3175495,31140751864621,1


# Extract out orders that are on time on first delivery attempt. Label those without second delivery attempt as on time. Then for the rest, check for time delta between two delivery attemps to decide whether an order is late (delta > 3).

In [22]:
orders_df_first_deli_ontime = orders_df[orders_df['ontime_first']]
orders_df_first_deli_ontime

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,origin,dest,sla,days_first,ontime_first
0,2215676524,2020-03-02 16:39:57,2020-03-05 13:09:18,NaT,"baging ldl buenavista,patag.cagayan buagsong,c...",pantranco vill. 417 warehouse# katipunan 532 (...,manila,manila,3,2,True
1,2219624609,2020-03-04 16:19:28,2020-03-06 10:53:56,2020-03-10 08:02:56,coloma's quzom casanas site1 masiyan 533a stol...,"bldg 210a moras c42b 2b16,168 church) complex ...",manila,manila,3,1,True
2,2220979489,2020-03-04 15:20:34,2020-03-06 09:56:19,NaT,"21-o lumangdaan,capitangan,abucay,bataan .bign...","#66 150-c, drive, milagros joe socorro metro m...",manila,manila,3,1,True
3,2221066352,2020-03-05 22:36:56,2020-03-07 12:45:41,NaT,"616espiritu martinville,manuyo #5paraiso kengi...","999maii 201,26 villaruel barretto gen.t number...",manila,manila,3,1,True
4,2222478803,2020-03-04 18:38:25,2020-03-06 15:41:40,NaT,l042 summerbreezee1 l2(balanay analyn lot760 c...,g66manila hiyas fitness maysilo magdiwang lt.4...,luzon,manila,5,1,True
...,...,...,...,...,...,...,...,...,...,...,...
3176308,31504087640510,2020-04-02 18:02:08,2020-04-04 13:22:22,NaT,"mayapis. edeliza jc274 5542 brgy.dita, metro m...",intimate st) compound. sekiat 2b8 98-b mel far...,manila,manila,3,1,True
3176309,31504147352227,2020-04-03 14:23:31,2020-04-06 21:17:27,2020-04-07 11:52:11,"superdome porky k307 libertad, pcgcc sub.bagui...","13th 3616 pilipinas, :1v26 1f-5 (village) bl10...",manila,manila,3,2,True
3176310,31504462290482,2020-04-02 19:29:11,2020-04-04 09:55:40,NaT,galay's 1republic 333-18 diliman 74d malbay fe...,"lane, 379b 1739a ivc afpfc cainta, waterpark, ...",manila,manila,3,1,True
3176311,31504851495943,2020-04-03 13:19:54,2020-04-07 11:19:12,NaT,"#4560, 80south biloca vi!!age km21, manresa, ,...",lrt-2 consolacion 224-a paradise 6d 36st champ...,luzon,manila,5,2,True


In [23]:
orders_df_first_final = orders_df_first_deli_ontime[pd.isnull(orders_df_first_deli_ontime['2nd_deliver_attempt'])]
orders_df_first_final

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,origin,dest,sla,days_first,ontime_first
0,2215676524,2020-03-02 16:39:57,2020-03-05 13:09:18,NaT,"baging ldl buenavista,patag.cagayan buagsong,c...",pantranco vill. 417 warehouse# katipunan 532 (...,manila,manila,3,2,True
2,2220979489,2020-03-04 15:20:34,2020-03-06 09:56:19,NaT,"21-o lumangdaan,capitangan,abucay,bataan .bign...","#66 150-c, drive, milagros joe socorro metro m...",manila,manila,3,1,True
3,2221066352,2020-03-05 22:36:56,2020-03-07 12:45:41,NaT,"616espiritu martinville,manuyo #5paraiso kengi...","999maii 201,26 villaruel barretto gen.t number...",manila,manila,3,1,True
4,2222478803,2020-03-04 18:38:25,2020-03-06 15:41:40,NaT,l042 summerbreezee1 l2(balanay analyn lot760 c...,g66manila hiyas fitness maysilo magdiwang lt.4...,luzon,manila,5,1,True
5,2222597288,2020-03-04 21:25:20,2020-03-07 15:41:49,NaT,c-846 park&shop opens valeda/sunrise) compuwar...,33 sta.mesa 1929 hawthorn metro manila,manila,manila,3,2,True
...,...,...,...,...,...,...,...,...,...,...,...
3176305,31502339045455,2020-04-03 20:55:15,2020-04-07 11:24:45,NaT,"napolan. auther 1,digos 3323-24, 1-ll consorti...","62, 2d (atm 462a arkong metro manila",manila,manila,3,2,True
3176307,31503788156046,2020-04-03 12:09:06,2020-04-04 12:17:10,NaT,"sweetberry ecija(mercury cailin matabao,buenav...",proper. alavarado bl-10 #3030 tarpaulin) 1115 ...,manila,manila,3,1,True
3176308,31504087640510,2020-04-02 18:02:08,2020-04-04 13:22:22,NaT,"mayapis. edeliza jc274 5542 brgy.dita, metro m...",intimate st) compound. sekiat 2b8 98-b mel far...,manila,manila,3,1,True
3176310,31504462290482,2020-04-02 19:29:11,2020-04-04 09:55:40,NaT,galay's 1republic 333-18 diliman 74d malbay fe...,"lane, 379b 1739a ivc afpfc cainta, waterpark, ...",manila,manila,3,1,True


In [24]:
good = pd.DataFrame({'orderid': orders_df_first_final['orderid'], 'is_late': 0})
good

,orderid,is_late
0,2215676524,0
2,2220979489,0
3,2221066352,0
4,2222478803,0
5,2222597288,0
...,...,...
3176305,31502339045455,0
3176307,31503788156046,0
3176308,31504087640510,0
3176310,31504462290482,0


In [25]:
result_df = result_df.append(good, ignore_index=True)
result_df

,orderid,is_late
0,2224695304,1
1,2227816770,1
2,2227831996,1
3,2227849732,1
4,2227878093,1
...,...,...
1932634,31502339045455,0
1932635,31503788156046,0
1932636,31504087640510,0
1932637,31504462290482,0


In [26]:
bad = orders_df_first_deli_ontime[~pd.isnull(orders_df_first_deli_ontime['2nd_deliver_attempt'])]
bad

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,origin,dest,sla,days_first,ontime_first
1,2219624609,2020-03-04 16:19:28,2020-03-06 10:53:56,2020-03-10 08:02:56,coloma's quzom casanas site1 masiyan 533a stol...,"bldg 210a moras c42b 2b16,168 church) complex ...",manila,manila,3,1,True
6,2222738456,2020-03-02 19:31:52,2020-03-05 13:22:32,2020-03-09 08:33:41,"ka-insong stockholme lhuilier, polanco. malen ...","salcedo midlands 980 wall, 16c paraiso mall（ph...",manila,manila,3,2,True
8,2224704587,2020-03-04 15:42:59,2020-03-05 14:14:26,2020-03-09 08:03:03,"lumber,naprada pot kadir bagyan blanga firends...",skies dept. crystal apello stockroom.star (dag...,luzon,manila,5,0,True
16,2227771109,2020-03-02 17:28:00,2020-03-04 11:24:05,2020-03-05 13:48:09,"ave,cor.matalino langsila -velez adnai st.rumh...","943-a e26-28 (katapat) 2662 303, 8floor salon,...",manila,manila,3,1,True
17,2227783105,2020-03-02 16:09:21,2020-03-05 13:46:47,2020-03-07 09:58:41,"adios obdulias drive,bacolod j45 (siony palind...",so mary ay caloy assisi 412b metro manila,manila,manila,3,2,True
...,...,...,...,...,...,...,...,...,...,...,...
3176293,31497388770741,2020-04-03 16:16:38,2020-04-06 12:12:16,2020-04-07 20:27:18,"flawless, paglabas, echivarri 0993 blk.51lot58...",(commonwealth) paang ugo guinto shopping.taft ...,manila,manila,3,1,True
3176298,31500223560475,2020-04-03 14:05:00,2020-04-06 16:26:40,2020-04-07 19:03:47,"motoccleparts a-lot29 namuag hal bakery,6005 s...",heva thursday gadget philippe escuela virgo pu...,manila,manila,3,2,True
3176306,31502490611412,2020-04-02 13:43:28,2020-04-04 12:14:56,2020-04-06 19:08:01,"ncm int.2n 3,baranggay 3275-d 337-g cor robins...","fass petra sora baba, financial (acct#3) subd....",luzon,manila,5,1,True
3176309,31504147352227,2020-04-03 14:23:31,2020-04-06 21:17:27,2020-04-07 11:52:11,"superdome porky k307 libertad, pcgcc sub.bagui...","13th 3616 pilipinas, :1v26 1f-5 (village) bl10...",manila,manila,3,2,True


In [27]:
bad['is_late'] = (bad.apply(lambda row: get_days(row['1st_deliver_attempt'], row['2nd_deliver_attempt']), axis=1) > 3).astype('int')
bad

C:\Users\Luca\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,origin,dest,sla,days_first,ontime_first,is_late
1,2219624609,2020-03-04 16:19:28,2020-03-06 10:53:56,2020-03-10 08:02:56,coloma's quzom casanas site1 masiyan 533a stol...,"bldg 210a moras c42b 2b16,168 church) complex ...",manila,manila,3,1,True,0
6,2222738456,2020-03-02 19:31:52,2020-03-05 13:22:32,2020-03-09 08:33:41,"ka-insong stockholme lhuilier, polanco. malen ...","salcedo midlands 980 wall, 16c paraiso mall（ph...",manila,manila,3,2,True,0
8,2224704587,2020-03-04 15:42:59,2020-03-05 14:14:26,2020-03-09 08:03:03,"lumber,naprada pot kadir bagyan blanga firends...",skies dept. crystal apello stockroom.star (dag...,luzon,manila,5,0,True,0
16,2227771109,2020-03-02 17:28:00,2020-03-04 11:24:05,2020-03-05 13:48:09,"ave,cor.matalino langsila -velez adnai st.rumh...","943-a e26-28 (katapat) 2662 303, 8floor salon,...",manila,manila,3,1,True,0
17,2227783105,2020-03-02 16:09:21,2020-03-05 13:46:47,2020-03-07 09:58:41,"adios obdulias drive,bacolod j45 (siony palind...",so mary ay caloy assisi 412b metro manila,manila,manila,3,2,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3176293,31497388770741,2020-04-03 16:16:38,2020-04-06 12:12:16,2020-04-07 20:27:18,"flawless, paglabas, echivarri 0993 blk.51lot58...",(commonwealth) paang ugo guinto shopping.taft ...,manila,manila,3,1,True,0
3176298,31500223560475,2020-04-03 14:05:00,2020-04-06 16:26:40,2020-04-07 19:03:47,"motoccleparts a-lot29 namuag hal bakery,6005 s...",heva thursday gadget philippe escuela virgo pu...,manila,manila,3,2,True,0
3176306,31502490611412,2020-04-02 13:43:28,2020-04-04 12:14:56,2020-04-06 19:08:01,"ncm int.2n 3,baranggay 3275-d 337-g cor robins...","fass petra sora baba, financial (acct#3) subd....",luzon,manila,5,1,True,0
3176309,31504147352227,2020-04-03 14:23:31,2020-04-06 21:17:27,2020-04-07 11:52:11,"superdome porky k307 libertad, pcgcc sub.bagui...","13th 3616 pilipinas, :1v26 1f-5 (village) bl10...",manila,manila,3,2,True,0


In [28]:
bad_result = bad.loc[:, ['orderid', 'is_late']]
bad_result

,orderid,is_late
1,2219624609,0
6,2222738456,0
8,2224704587,0
16,2227771109,0
17,2227783105,0
...,...,...
3176293,31497388770741,0
3176298,31500223560475,0
3176306,31502490611412,0
3176309,31504147352227,0


In [29]:
result_df = result_df.append(bad_result, ignore_index=True)
result_df

,orderid,is_late
0,2224695304,1
1,2227816770,1
2,2227831996,1
3,2227849732,1
4,2227878093,1
...,...,...
3176308,31497388770741,0
3176309,31500223560475,0
3176310,31502490611412,0
3176311,31504147352227,0


In [30]:
result_df.to_csv('output.csv', index=False)